# **Machine Learning para Business Intelligence** 
#### Profesor: Lucas Gómez Tobón

# Orderbot

In [1]:
import openai
import panel as pn
pn.extension()

In [3]:
# Cargar su API Key
openai.api_key = ''

In [9]:
def get_completion_from_messages(messages, model = "gpt-3.5-turbo", temperature = 0):
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        # this is the degree of randomness of the model's output
        temperature = temperature, 
    )
    return response.choices[0].message["content"]

In [10]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content': f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width = 600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width = 600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [11]:
# collect display 
panels = [] 

In [12]:
# Contexto es una lista encargada de acumular los mensajes!
context = [
    {'role': 'system', 'content': """
Eres OrderBot, un servicio automatizado para recibir pedidos para un restaurante 
de pizza.

Primero saludas al cliente, luego recibe el pedido y después preguntas si es para 
recoger en el restaurante o a domicilio.

Esperas a recoger el pedido completo, luego lo resumirás y verificarás por última 
vez si el cliente quiere agregar algo más. Para verificar si el cliente desea algo más, sugierele 
alguna bebida en caso de que no haya solicitado ya algo de tomar. En caso de que ya tenga una bebida,
podrías sugerirle alguna de las adiciones.

Si es a domicilio, pides una dirección.

Finalmente, recoges el pago.

Asegúrate de aclarar todas las opciones, extras y tamaños para identificar de manera
única el artículo del menú. 

Adicionalmente, recuerda que solo estan disponibles las pizzas del menú, no es posible ordenar alguna 
diferente.

Responderás de manera corta, muy amigable y conversacional.

El menú incluye:
pizza de pepperoni 12.95, 10.00, 7.00
pizza de queso 10.95, 9.25, 6.50
pizza de berenjena 11.95, 9.75, 6.75
papas fritas 4.50, 3.50
ensalada griega 7.25

Adicionales:
extra de queso 2.00
champiñones 1.50
salchicha 3.00
tocino canadiense 3.50
salsa AI 1.50
pimientos 1.00

Bebidas:
coca cola 3.00, 2.00, 1.00
sprite 3.00, 2.00, 1.00
agua embotellada 5.00
"""} ]  

In [13]:
inp = pn.widgets.TextInput(value = "Hola!", placeholder = 'Escribe tu pedido')
button_conversation = pn.widgets.Button(name = "Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator = True, height = 300),
)

In [14]:
dashboard

Column
    [0] TextInput(placeholder='Escribe tu pedido')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [15]:
messages = context.copy()
messages.append(
{'role':'system', 'content': 'Crear un resumen de la orden anterior en un JSON. Enumerar el precio de cada artículo. \
Los campos deben ser: 1) pizza, incluir tamaño 2) lista de ingredientes 3) lista de bebidas, incluir tamaño 4) lista de acompañamientos, incluir tamaño 5) precio total.'},    
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

Aquí está el resumen de tu pedido en formato JSON:

{
  "pizza": [
    {
      "tipo": "Pepperoni",
      "tamaño": "Personal",
      "precio": 7.00
    },
    {
      "tipo": "Hawaiana",
      "tamaño": "Grande",
      "precio": 12.95
    }
  ],
  "bebidas": [],
  "acompañamientos": [],
  "precio_total": 19.95
}

Espero que esto te sea útil. ¡Gracias por ordenar con nosotros!
